In [1]:
import numpy as np
from main_grape.Grape import Grape
from main_grape.Evolve import Evolve
from math_functions.helper_fns import *
%pylab inline




Populating the interactive namespace from numpy and matplotlib


In [2]:
import scipy.io as sio

a = sio.loadmat('File.mat')


In [ ]:

#Defining time scales
total_time = 8.0
steps = 300
qubit_num = 5
qubit_state_num = 2

       

#Defining Forbidden sates

#states_forbidden_list = []

#Defining Concerned states (starting states)
psi0 = concerned(qubit_num,qubit_state_num)

#Defining states to include in the drawing of occupation
states_draw_list = range(qubit_state_num**qubit_num)
states_draw_names =[]
for ii in states_draw_list:
    states_draw_names.append(Basis(ii,qubit_num,qubit_state_num))

#Defining U (Target)

U= transmon_gate(qft(qubit_num),qubit_state_num)
#U = np.diag(np.arange(0,4))

#Defining U0 (Initial)
qq_identity = np.identity(qubit_state_num**qubit_num)
U0= qq_identity

#Defining control Hs
Q_x   = np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)+np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1)
Q_y   = (0+1j) *(np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)-np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1))
Q_z   = np.diag(np.arange(0,qubit_state_num))
Q_I = np.identity(qubit_state_num)

Q_xx = multi_kron(Q_x,qubit_num)
Q_yy = multi_kron(Q_y,qubit_num)
Q_zz = multi_kron(Q_z,qubit_num)

H0 = 0.5* (Q_xx+Q_yy+Q_zz)

Hops = []
Hnames = []
ops_max_amp = []
Hops,Hnames,ops_max_amp = append_separate_krons(Q_x,'x',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=1.1)
Hops,Hnames,ops_max_amp = append_separate_krons(Q_y,'y',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=1.1)
Hops,Hnames,ops_max_amp = append_separate_krons(Q_z,'z',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=1.1)
Hops.extend([Q_xx,Q_yy,Q_zz])
Hnames.extend(['all x','all y','all z'])
ops_max_amp.extend([1.1,1.1,1.1])

#Defining convergence parameters
max_iterations = 2000
decay = max_iterations/2
convergence = {'rate':0.05, 'update_step':10, 'max_iterations':max_iterations,\
               'conv_target':1e-4,'learning_rate_decay':decay}
reg_coeffs = {'alpha' : 0, 'z':0, 'dwdt':0,'d2wdt2':0, 'inter':0}
#multi_mode = {'dressed':dressed, 'vectors':v_c, 'qnum':qubit_state_num, 'mnum': mode_state_num,\
              #'f':freq_ge, 'es':w_c, 'g1':qm_g1, 'D':D, 'Interpolation':True, 'Modulation':True}

print states_draw_names
print Hnames
print np.shape(U)
print np.shape(Hops)





['00000', '00001', '00010', '00011', '00100', '00101', '00110', '00111', '01000', '01001', '01010', '01011', '01100', '01101', '01110', '01111', '10000', '10001', '10010', '10011', '10100', '10101', '10110', '10111', '11000', '11001', '11010', '11011', '11100', '11101', '11110', '11111']
['xiiii', 'ixiii', 'iixii', 'iiixi', 'iiiix', 'yiiii', 'iyiii', 'iiyii', 'iiiyi', 'iiiiy', 'ziiii', 'iziii', 'iizii', 'iiizi', 'iiiiz', 'all x', 'all y', 'all z']
(32, 32)
(18, 32, 32)


In [ ]:
 #Extra options
#reg_coeffs = reg_coeffs,draw = [states_draw_list,states_draw_names] ,forbidden = states_forbidden_list,multi_mode = multi_mode,
#maxA = ops_max_amp,use_gpu = True
uks,Uf=Grape(H0,Hops,Hnames,U,U0,total_time,steps,psi0,convergence,reg_coeffs = reg_coeffs,
      draw = [states_draw_list,states_draw_names], show_plots = False, method = 'L-BFGS-B', Unitary_error = 1e-4,switch=True)

Using 6 Taylor terms and 1 Scaling & Squaring terms
Building graph:
State initialized.
Operators initialized.
Operators weight initialized.
Intermediate states initialized.
Propagator initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.

In [ ]:
from qutip import *
import qutip.control.pulseoptim as cpo
import qutip.control.pulsegen as pulsegen
import qutip.logging as logging
logger = logging.get_logger()
#Set this to None or logging.WARN for 'quiet' execution
log_level = logging.INFO
size = len(H0)
H_d = Qobj(H0)
H_c = []
for op in Hops:
    H_c.append(Qobj(op))
U_0 = identity(size)
U_targ = Qobj(U)
n_ts = steps
n_ts = 90

evo_time = total_time
fid_err_targ = convergence['conv_target']
fid_err_targ = 4e-5
# Maximum iterations for the optisation algorithm
max_iter = 1000 #Can change
# Maximum (elapsed) time allowed in seconds
max_wall_time = 30000
# Minimum gradient (sum of gradients squared)
# as this tends to 0 -> local minima has been found
min_grad = 1e-20
p_type = 'RND'
f_ext = "{}_n_ts{}_ptype{}.txt".format('test', n_ts, p_type)
result = cpo.optimize_pulse_unitary(H_d, H_c, U_0, U_targ, n_ts, evo_time, 
                fid_err_targ=fid_err_targ, min_grad=min_grad, 
                max_iter=max_iter, max_wall_time=max_wall_time, 
                out_file_ext=f_ext, init_pulse_type=p_type, 
                log_level=log_level, gen_stats=True)
result.stats.report()

print("********* Summary *****************")
print("Final fidelity error {}".format(result.fid_err))
print("Final gradient normal {}".format(result.grad_norm_final))
print("Terminated due to {}".format(result.termination_reason))
print("Number of iterations {}".format(result.num_iter))
print("Completed in {} HH:MM:SS.US".format(
        datetime.timedelta(seconds=result.wall_time)))

In [ ]:

file = open('ctrl_amps_final_test_n_ts'+str(n_ts)+'_ptype'+str(p_type)+'.txt', 'r')

    
u=[]
uz=[]
u0=[]
for line in file:
    a=line.split()
    for ii in range (len(Hops)):
        u.append(float(a[ii+1]))
print np.shape(u)       
for ii in range (len(Hops)):
    ui=[]
    for jj in range (n_ts):
        index = ii+len(Hops)*jj
        ui.append(u[index])
    u0.append(np.reshape(ui,[n_ts]))
#u0.append(ux)
#u0.append(uz)
print np.shape(u0)
plt.figure(figsize=(18,6))
for jj in range (len(u0)):
    plt.plot(np.array([total_time* ii/steps for ii in range(n_ts)]),np.array(u0[jj]),label='u'+Hnames[jj])
plt.title('Optimized pulse')
            
plt.ylabel('Amplitude')
plt.xlabel('Time (ns)')
plt.legend()

Evolve(H0,Hops,U0,total_time,n_ts,psi0, u0, draw = [states_draw_list,states_draw_names], U = U,Hnames = Hnames)
